In [ ]:
from langchain_community.llms import Ollama
import pandas as pd
from datetime import datetime, timedelta
import os
import json

In [ ]:
# articles done. 
# master_df = pd.DataFrame()
# check for frequencies of each company too
master_di = {}
# s&p symbols
spsymbols = set(pd.read_csv('s&p.csv')['Symbol'].tolist())

di_by_symbol = {}

for day in os.listdir('articles'):
    # read article json file
    print(day)
    day_di = {}

    # read the json file
    with open(f'articles/{day}', 'r') as f:
        data = json.load(f)

        for article in data:
            # we';re interested in the headline and symbols and date (date that is being analyzed) fields
            headline, symbols = article['headline'], article['symbols']

            # add to master di
            for symbol in symbols:
                if symbol not in day_di and symbol in spsymbols: # only add if it's in the s&p
                    day_di[symbol] = [headline]
                elif symbol in day_di and symbol in spsymbols: # add if it's in s&p and already in the di
                    day_di[symbol].append(headline)


    # add to master di
    
    master_di[day.split('.')[0]] = day_di
    di_by_symbol.update(day_di)

"""
now, master di looks like the following:
{
    '2021-01-01': {
        'AAPL': ['headline1', 'headline2', ...],
        'GOOGL': ['headline1', 'headline2', ...],
        ...
    },
    '2021-01-02': {
        'AAPL': ['headline1', 'headline2', ...],
        'GOOGL': ['headline1', 'headline2', ...],
        ...
    },
    ...
}
"""




In [ ]:
print(len(master_di))

In [ ]:
# tally number of articles for each company across all dates
company_freq = {}
for date in master_di:
    for company in master_di[date]:
        if company not in company_freq:
            company_freq[company] = len(master_di[date][company])
        else:
            company_freq[company] += len(master_di[date][company])

# sort the companies by frequency
company_freq = dict(sorted(company_freq.items(), key=lambda x: x[1], reverse=True))
print(company_freq)

In [34]:
llm = Ollama(model="llama3", cache=False, top_p = 0.2, top_k = 3, temperature=0) # need to get as deterministic as possible
# top k small means that we are more deterministic
# top p small means that we are more deterministic
#intense coding right here

In [44]:
import pandas as pd
import re


master_sentiment = {}
failed = {}
misaligned = {}
for date in master_di:
    print("PARSING DATE", date)
    
    for idx, current_ticker in enumerate(master_di[date]):
        print("PARSING TICKER", idx, current_ticker)
        headlines = []
 
        for headline in master_di[date][current_ticker]:

            # st = "{\"TSLA\": \"positive\", \"AAPL\": \"neutral\", \"GOOG\": \"negative\"} or {} if no companies mentioned"
            # prompt = f"""
            # The following financial news headline is about {current_ticker} but may mention other companies.
            # Please provide the sentiment (positive, neutral, or negative) solely in relation to {current_ticker} 
            # given the following headline in the brackets below. The sentiment should reflect the favorability of {current_ticker} for investors. 
            # If you are not sure, answer neutral.
            # [{headline}]
            # """.strip()
            # Omit any explanation of sentiment analysis.
            # You are a financial analyst with knowledge of all stock tickers in the s&p 500. 
            # for chunks in llm.stream(prompt):
            #     print(chunks, end="")
            headlines.append(headline)

        prompt = f"""
        you will be given a set of news headlines about a certain stock separated by bars and required to determine the sentiment in relation to that stock. 
        For example, the format will be headline one | headline two | headline three... 
        Determine the sentiment of each headline ("POSITIVE", "NEUTRAL", or "NEGATIVE") in relation to the provided stock ticker, 
        where "NEUTRAL" should be used for ambiguous headlines which do not explicitly mention sentiment of the company of interest. 
        You are forbidden from outputting explanations. 
        Only use headlines to determine sentiment and do not assume the content of the article.
        Verify that the output length is the same as number of headlines you received.
        
        Example output: ["POSITIVE", "NEGATIVE", "NEUTRAL"]. 
        
        Ticker: {current_ticker}
        Headlines: {"| ".join(headlines)}
        
        """
        # print(prompt)

        # reay what we need to do after this is get it into csv form, along with the percent change that happens over that time (not gotten yet)
        val = llm.invoke(prompt)
        # use regex to parse for dictionary, denoted by curly braces
        # print("**HEADLINE**")
        # print(headline)
        print(len(headlines))
        print("**RESPONSE**")
        print(val)
        try:
            li = eval(val)
        except:
            failed[date] = {"llm output": val, "headlines": headlines, "ticker": current_ticker}
            continue

        print("length of response", len(li), "length of headlines", len(headlines))
        if len(li) != len(headlines):
            misaligned[date] = f"response {len(li)} headlines {len(headlines)}"
            # continue
        # yeah it would be the same format as master_di (see above)
        #maybe a nested dict Ben? 

        # count frequencyh of positive, negative, neutral
        pos, neg, neu = 0, 0, 0
        for sentiment in li:
            sentiment = sentiment.upper()
            if sentiment == "POSITIVE":
                pos += 1
            elif sentiment == "NEGATIVE":
                neg += 1
            elif sentiment == "NEUTRAL":
                neu += 1

        master_sentiment[date] = {
            current_ticker: {
                'positive': pos,
                'negative': neg,
                'neutral': neu,
                'aggregated': (pos - neg) / (pos + neg + neu) # this is the "sentiment score
            }
        }

        
    

PARSING DATE 2015-01-06
PARSING TICKER 0 LLY
2
**RESPONSE**
["POSITIVE", "NEUTRAL"]
length of response 2 length of headlines 2
PARSING TICKER 1 AAL
1
**RESPONSE**
["POSITIVE"]
length of response 1 length of headlines 1
PARSING TICKER 2 AAPL
9
**RESPONSE**
["POSITIVE", "POSITIVE", "NEGATIVE", "NEUTRAL", "NEUTRAL", "NEUTRAL", "NEUTRAL", "NEUTRAL"]
length of response 8 length of headlines 9
PARSING TICKER 3 IBM
3
**RESPONSE**
["POSITIVE", "NEUTRAL", "NEUTRAL"]
length of response 3 length of headlines 3
PARSING TICKER 4 INTC
3
**RESPONSE**
["POSITIVE", "NEUTRAL", "POSITIVE"]
length of response 3 length of headlines 3
PARSING TICKER 5 WYNN
3
**RESPONSE**
["POSITIVE", "NEGATIVE", "NEUTRAL"]
length of response 3 length of headlines 3
PARSING TICKER 6 KO
1
**RESPONSE**
["POSITIVE"]
length of response 1 length of headlines 1
PARSING TICKER 7 HON
1
**RESPONSE**
["POSITIVE"]
length of response 1 length of headlines 1
PARSING TICKER 8 CHTR
1
**RESPONSE**
["POSITIVE"]
length of response 1 length of

In [ ]:
# for company in 

In [ ]:
#definition of spaghetti code -- creates a 2d list with top 5 tickers from each day in order [[aapl,nvida,amzn,amd,cost],[nvda,nvda,nvda,nvda,nvda]]
master_list = []
for date in master_sentiment:
    # max_num = [0,0,0,0,0]
    minMax = 0
    # max_ticker = ["","","","",""]
    ticker_to_sentiment = {}

    for ticker in master_sentiment[date]:
        if(master_sentiment[date][ticker]["aggregated"] > minMax):
            # max_num[max_num.index(min(max_num))] = master_sentiment[date][ticker]["aggregated"]
            ticker_to_sentiment[ticker] = master_sentiment[date][ticker]["aggregated"]
            minMax = min(ticker_to_sentiment.values())

    master_list.append(ticker_to_sentiment)